In [1]:
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
import json
import time
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [2]:
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
image_pred = pd.read_csv("image_predictions.tsv", sep='\t')

In [3]:
twitter_archive.head(20)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0   892420643555336193                    NaN                  NaN   
1   892177421306343426                    NaN                  NaN   
2   891815181378084864                    NaN                  NaN   
3   891689557279858688                    NaN                  NaN   
4   891327558926688256                    NaN                  NaN   
5   891087950875897856                    NaN                  NaN   
6   890971913173991426                    NaN                  NaN   
7   890729181411237888                    NaN                  NaN   
8   890609185150312448                    NaN                  NaN   
9   890240255349198849                    NaN                  NaN   
10  890006608113172480                    NaN                  NaN   
11  889880896479866881                    NaN                  NaN   
12  889665388333682689                    NaN                  NaN   
13  889638837579907072                    NaN                  NaN   
14  889531135344209921                    NaN                  NaN   
15  889278841981685760                    NaN                  NaN   
16  888917238123831296                    NaN                  NaN   
17  888804989199671297                    NaN                  NaN   
18  888554962724278272                    NaN                  NaN   
19  888202515573088257                    NaN                  NaN   

                    timestamp  \
0   2017-08-01 16:23:56 +0000   
1   2017-08-01 00:17:27 +0000   
2   2017-07-31 00:18:03 +0000   
3   2017-07-30 15:58:51 +0000   
4   2017-07-29 16:00:24 +0000   
5   2017-07-29 00:08:17 +0000   
6   2017-07-28 16:27:12 +0000   
7   2017-07-28 00:22:40 +0000   
8   2017-07-27 16:25:51 +0000   
9   2017-07-26 15:59:51 +0000   
10  2017-07-26 00:31:25 +0000   
11  2017-07-25 16:11:53 +0000   
12  2017-07-25 01:55:32 +0000   
13  2017-07-25 00:10:02 +0000   
14  2017-07-24 17:02:04 +0000   
15  2017-07-24 00:19:32 +0000   
16  2017-07-23 00:22:39 +0000   
17  2017-07-22 16:56:37 +0000   
18  2017-07-22 00:23:06 +0000   
19  2017-07-21 01:02:36 +0000   

                                               source  \
0   <a href="http://twitter.com/download/iphone" r...   
1   <a href="http://twitter.com/download/iphone" r...   
2   <a href="http://twitter.com/download/iphone" r...   
3   <a href="http://twitter.com/download/iphone" r...   
4   <a href="http://twitter.com/download/iphone" r...   
5   <a href="http://twitter.com/download/iphone" r...   
6   <a href="http://twitter.com/download/iphone" r...   
7   <a href="http://twitter.com/download/iphone" r...   
8   <a href="http://twitter.com/download/iphone" r...   
9   <a href="http://twitter.com/download/iphone" r...   
10  <a href="http://twitter.com/download/iphone" r...   
11  <a href="http://twitter.com/download/iphone" r...   
12  <a href="http://twitter.com/download/iphone" r...   
13  <a href="http://twitter.com/download/iphone" r...   
14  <a href="http://twitter.com/download/iphone" r...   
15  <a href="http://twitter.com/download/iphone" r...   
16  <a href="http://twitter.com/download/iphone" r...   
17  <a href="http://twitter.com/download/iphone" r...   
18  <a href="http://twitter.com/download/iphone" r...   
19  <a href="http://twitter.com/download/iphone" r...   

                                                 text  retweeted_status_id  \
0   This is Phineas. He's a mystical boy. Only eve...                  NaN   
1   This is Tilly. She's just checking pup on you....                  NaN   
2   This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3   This is Darla. She commenced a snooze mid meal...                  NaN   
4   This is Franklin. He would like you to stop ca...                  NaN   
5   Here we have a majestic great white breaching ...                  NaN   
6   Meet Jax. He enjoys ice cream so much he gets ...                  NaN   
7   When you watch your own

In [4]:
twitter_archive.text.sample()

1405    "I'm bathing the children what do you want?"  ...
Name: text, dtype: object

In [5]:
twitter_archive.tail()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
2351  666049248165822465                    NaN                  NaN   
2352  666044226329800704                    NaN                  NaN   
2353  666033412701032449                    NaN                  NaN   
2354  666029285002620928                    NaN                  NaN   
2355  666020888022790149                    NaN                  NaN   

                      timestamp  \
2351  2015-11-16 00:24:50 +0000   
2352  2015-11-16 00:04:52 +0000   
2353  2015-11-15 23:21:54 +0000   
2354  2015-11-15 23:05:30 +0000   
2355  2015-11-15 22:32:08 +0000   

                                                 source  \
2351  <a href="http://twitter.com/download/iphone" r...   
2352  <a href="http://twitter.com/download/iphone" r...   
2353  <a href="http://twitter.com/download/iphone" r...   
2354  <a href="http://twitter.com/download/iphone" r...   
2355  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
2351  Here we have a 1949 1st generation vulpix. Enj...                  NaN   
2352  This is a purebred Piers Morgan. Loves to Netf...                  NaN   
2353  Here is a very happy pup. Big fan of well-main...                  NaN   
2354  This is a western brown Mitsubishi terrier. Up...                  NaN   
2355  Here we have a Japanese Irish Setter. Lost eye...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
2351                       NaN                        NaN   
2352                       NaN                        NaN   
2353                       NaN                        NaN   
2354                       NaN                        NaN   
2355                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
2351  https://twitter.com/dog_rates/status/666049248...                 5   
2352  https://twitter.com/dog_rates/status/666044226...                 6   
2353  https://twitter.com/dog_rates/status/666033412...                 9   
2354  https://twitter.com/dog_rates/status/666029285...                 7   
2355  https://twitter.com/dog_rates/status/666020888...                 8   

      rating_denominator  name doggo floofer pupper puppo  
2351                  10  None  None    None   None  None  
2352                  10     a  None    None   None  None  
2353                  10     a  None    None   None  None  
2354                  10     a  None    None   None  None  
2355                  10  None  None    None   None  None

In [6]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [7]:
twitter_archive[~twitter_archive.retweeted_status_id.isnull()]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
19    888202515573088257                    NaN                  NaN   
32    886054160059072513                    NaN                  NaN   
36    885311592912609280                    NaN                  NaN   
68    879130579576475649                    NaN                  NaN   
73    878404777348136964                    NaN                  NaN   
74    878316110768087041                    NaN                  NaN   
78    877611172832227328                    NaN                  NaN   
91    874434818259525634                    NaN                  NaN   
95    873697596434513921                    NaN                  NaN   
97    873337748698140672                    NaN                  NaN   
101   872668790621863937                    NaN                  NaN   
109   871166179821445120                    NaN                  NaN   
118   869988702071779329                    NaN                  NaN   
124   868639477480148993                    NaN                  NaN   
130   867072653475098625                    NaN                  NaN   
132   866816280283807744                    NaN                  NaN   
137   866094527597207552                    NaN                  NaN   
146   863471782782697472                    NaN                  NaN   
155   861769973181624320                    NaN                  NaN   
159   860981674716409858                    NaN                  NaN   
160   860924035999428608                    NaN                  NaN   
165   860177593139703809                    NaN                  NaN   
171   858860390427611136                    NaN                  NaN   
180   857062103051644929                    NaN                  NaN   
182   856602993587888130                    NaN                  NaN   
185   856330835276025856                    NaN                  NaN   
194   855245323840757760                    NaN                  NaN   
195   855138241867124737                    NaN                  NaN   
204   852936405516943360                    NaN                  NaN   
211   851953902622658560                    NaN                  NaN   
...                  ...                    ...                  ...   
784   775096608509886464                    NaN                  NaN   
794   773336787167145985                    NaN                  NaN   
800   772615324260794368                    NaN                  NaN   
811   771171053431250945                    NaN                  NaN   
815   771004394259247104                    NaN                  NaN   
818   770743923962707968                    NaN                  NaN   
822   770093767776997377                    NaN                  NaN   
826   769335591808995329                    NaN                  NaN   
829   768909767477751808                    NaN                  NaN   
833   768554158521745409                    NaN                  NaN   
841   766864461642756096                    NaN                  NaN   
847   766078092750233600                    NaN                  NaN   
860   763167063695355904                    NaN                  NaN   
868   761750502866649088                    NaN                  NaN   
872   761371037149827077                    NaN                  NaN   
885   760153949710192640                    NaN                  NaN   
890   759566828574212096                    NaN                  NaN   
895   759159934323924993                    NaN                  NaN   
908   757729163776290825                    NaN                  NaN   
911   757597904299253760                    NaN                  NaN   
926   754874841593970688                    NaN                  NaN   
937   753298634498793472                    NaN                  NaN   
943   752701944171524096                    NaN                  NaN   
949   752309394570878976                    NaN         

In [8]:
image_pred.head(5)

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

In [9]:
image_pred.tail()

tweet_id                                          jpg_url  \
2070  891327558926688256  https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg   
2071  891689557279858688  https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg   
2072  891815181378084864  https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg   
2073  892177421306343426  https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg   
2074  892420643555336193  https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg   

      img_num           p1   p1_conf  p1_dog                  p2   p2_conf  \
2070        2       basset  0.555712    True    English_springer  0.225770   
2071        1  paper_towel  0.170278   False  Labrador_retriever  0.168086   
2072        1    Chihuahua  0.716012    True            malamute  0.078253   
2073        1    Chihuahua  0.323581    True            Pekinese  0.090647   
2074        1       orange  0.097049   False               bagel  0.085851   

      p2_dog                           p3   p3_conf  p3_dog  
2070    True  German_short-haired_pointer  0.175219    True  
2071    True                      spatula  0.040836   False  
2072    True                       kelpie  0.031379    True  
2073    True                     papillon  0.068957    True  
2074   False                       banana  0.076110   False

In [10]:
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [11]:
with open("tweet_json.txt") as file:
    twt_data = json.load(file)
tweet_info = pd.DataFrame(columns=["tweet_id", "timestamp", "retweet_count", "favorite_count"])
for twt in twt_data["tweets"]:
    tweet_info = tweet_info.append({"tweet_id":twt['id'], "timestamp":twt['created_at']\
                                   ,"retweet_count":twt['retweet_count'], "favorite_count":\
                                    twt['favorite_count']},ignore_index = True) 
    

In [12]:
tweet_info.head(5)

tweet_id                       timestamp retweet_count  \
0  892420643555336193  Tue Aug 01 16:23:56 +0000 2017          8330   
1  892177421306343426  Tue Aug 01 00:17:27 +0000 2017          6154   
2  891815181378084864  Mon Jul 31 00:18:03 +0000 2017          4072   
3  891689557279858688  Sun Jul 30 15:58:51 +0000 2017          8474   
4  891327558926688256  Sat Jul 29 16:00:24 +0000 2017          9168   

  favorite_count  
0          38100  
1          32690  
2          24599  
3          41456  
4          39628

In [13]:
tweet_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 4 columns):
tweet_id          2340 non-null object
timestamp         2340 non-null object
retweet_count     2340 non-null object
favorite_count    2340 non-null object
dtypes: object(4)
memory usage: 73.2+ KB


# Assess

## Quality Issues


#### `twitter_archive` table
<ul>
    <li>timestamp to date time data type</li>
    <li>retweeted_status_timestamp is an object rather than data_time data type.</li>
    <li>Symbols in text column</li>
    <li>tweet_id to be converted to string data type from integer</li>
    <li>in_reply_to_status_id to String data type</li>
    <li>in_reply_to_user_id to String data type</li>
    
</ul>

#### `image_pred` table
<ul>
    <li>"a" tags in source</li>    
    <li>"_" used as separator in p1 column of image_pred df</li>
    <li>"_" used as separator in p2 column of image_pred df</li>
    <li>"_" used as separator in p3 column of image_pred df</li>
    <li>tweet_id in image_pred to be converted to strint data type from integer</li>
    <li>Types of dogs to category data type</li>
    <li>image_num to String data type</li>
    <li>desktop_computer as value in p1</li>
    <li> "-" in certain letters to split them</li>
    <li>Not all first names, second names and third names are capitalized</li>
</ul>

#### `tweet_info` table
<ul>
    <li>timestamp should be date time data type</li>
    <li>retweet_count column should be integer</li>
    <li>favorite_count should be integer</li>
</ul>

## Tidiness Issues

<ul>
    <li>Type of dog in "doggo", "floofer", "pupper", "puppo" columns instead of one column.</li>
    <li>`tweet_info` table should be a part of `twitter_archive` table</li>
</ul>

# Clean

In [14]:
twitter_clean = twitter_archive.copy()
image_pred_clean = image_pred.copy()
tweet_info_clean = tweet_info.copy()

### `twitter_archive`: timestamp and retweeted_status_timestamp to date time data type

#### Define

timestamp column and retweeted_status_timestamp column is in object data type and changing it to date_time date type makes analysis easier

#### Code

In [15]:
twitter_clean["timestamp"] = pd.to_datetime(twitter_archive["timestamp"])
twitter_clean["retweeted_status_timestamp"] = pd.to_datetime(twitter_clean["retweeted_status_timestamp"])

#### Test

In [16]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime

### `image_pred`: HTML a tags in source column

#### Define

URL links are enclosed inside opening and closing "a" tags.

#### Code

In [17]:
twitter_clean["source"] = twitter_clean["source"].str.extract(r'(http[^\"^\s]*)')

#### Test

In [18]:
twitter_clean.source.sample(10)

738     http://twitter.com/download/iphone
1455    http://twitter.com/download/iphone
1472    http://twitter.com/download/iphone
452     http://twitter.com/download/iphone
2115    http://twitter.com/download/iphone
2077    http://twitter.com/download/iphone
1070    http://twitter.com/download/iphone
1092    http://twitter.com/download/iphone
793     http://twitter.com/download/iphone
1564    http://twitter.com/download/iphone
Name: source, dtype: object

In [19]:
tweet_info.head(5)

tweet_id                       timestamp retweet_count  \
0  892420643555336193  Tue Aug 01 16:23:56 +0000 2017          8330   
1  892177421306343426  Tue Aug 01 00:17:27 +0000 2017          6154   
2  891815181378084864  Mon Jul 31 00:18:03 +0000 2017          4072   
3  891689557279858688  Sun Jul 30 15:58:51 +0000 2017          8474   
4  891327558926688256  Sat Jul 29 16:00:24 +0000 2017          9168   

  favorite_count  
0          38100  
1          32690  
2          24599  
3          41456  
4          39628

### `tweet_info` tweet id is an object

#### Define

Convert tweet_id column to integer data type

#### Code

In [20]:
tweet_info_clean["tweet_id"] = tweet_info_clean.tweet_id.astype(int)

#### Test

In [21]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime

### `twitter_archive` tweet_id should be a string

#### Define
Convert tweet_id to String

### Code

In [22]:
twitter_clean["tweet_id"] = twitter_clean.tweet_id.astype(str)

#### Test

In [23]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetim

### `image_pred` "_" used as separators in p1, p2 and p3

#### Define
Replace _ with spaces in p1, p2 and p3 columns

#### Code

In [24]:
image_pred_clean.p1.head()

0    Welsh_springer_spaniel
1                   redbone
2           German_shepherd
3       Rhodesian_ridgeback
4        miniature_pinscher
Name: p1, dtype: object

In [25]:
image_pred_clean["p1"] = image_pred_clean.p1.str.replace("_", " ")
image_pred_clean["p2"] = image_pred_clean.p2.str.replace("_", " ")
image_pred_clean["p3"] = image_pred_clean.p3.str.replace("_", " ")

#### Test

In [26]:
print(image_pred_clean.p1.head())
print(image_pred_clean.p2.head())
print(image_pred_clean.p3.head())

0    Welsh springer spaniel
1                   redbone
2           German shepherd
3       Rhodesian ridgeback
4        miniature pinscher
Name: p1, dtype: object
0                collie
1    miniature pinscher
2              malinois
3               redbone
4            Rottweiler
Name: p2, dtype: object
0      Shetland sheepdog
1    Rhodesian ridgeback
2             bloodhound
3     miniature pinscher
4               Doberman
Name: p3, dtype: object


### `image_pred` "-" used as separator of names in some rows

#### Define

Replace - with spaces in p1, p2 and p3 colunns

#### Code

In [27]:
image_pred_clean["p1"] = image_pred_clean.p1.str.replace("-", " ")
image_pred_clean["p2"] = image_pred_clean.p2.str.replace("-", " ")
image_pred_clean["p3"] = image_pred_clean.p3.str.replace("-", " ")

#### Test

In [28]:
image_pred_clean.p1.sample(10)

337           killer whale
51                    swab
174             comic book
372                 wombat
17                     hen
347               terrapin
589     Scottish deerhound
696              Chihuahua
1502              malamute
961                 collie
Name: p1, dtype: object

### `image_pred` Not all first names, second names and third names are capitalized

#### Define

Capitalize the first letters of all rows in p1, p2 and p3 columns

#### Code

In [29]:
image_pred_clean["p1"] = image_pred_clean.p1.str.title()
image_pred_clean["p2"] = image_pred_clean.p2.str.title()
image_pred_clean["p3"] = image_pred_clean.p3.str.title()

#### Test

In [30]:
print(image_pred_clean.p1.sample(5))
print(image_pred_clean.p2.sample(5))
print(image_pred_clean.p3.sample(5))

1389      Keeshond
1978      Pembroke
1808    Eskimo Dog
117         Angora
1876       Samoyed
Name: p1, dtype: object
1777    Labrador Retriever
1520        French Bulldog
1887                   Cup
1477    Labrador Retriever
637       Miniature Poodle
Name: p2, dtype: object
507     American Staffordshire Terrier
393                             Borzoi
745                    Tibetan Mastiff
536                     Great Pyrenees
1816                  Golden Retriever
Name: p3, dtype: object


### `tweet_info` retweet count is an object

#### Define

Convert retweet count column to integer data type

#### Code

In [31]:
tweet_info_clean["retweet_count"] = tweet_info_clean.retweet_count.astype(int)

#### Test

In [32]:
tweet_info_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 4 columns):
tweet_id          2340 non-null int64
timestamp         2340 non-null object
retweet_count     2340 non-null int64
favorite_count    2340 non-null object
dtypes: int64(2), object(2)
memory usage: 73.2+ KB


### `tweet_info` tweet_id is an integer

#### Define

Convert tweet_id to String data type

#### Code

In [33]:
tweet_info_clean["tweet_id"] = tweet_info_clean.tweet_id.astype(str)

#### Test

In [34]:
tweet_info_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 4 columns):
tweet_id          2340 non-null object
timestamp         2340 non-null object
retweet_count     2340 non-null int64
favorite_count    2340 non-null object
dtypes: int64(1), object(3)
memory usage: 73.2+ KB


### `tweet_info` favorite count is an object

#### Define

Convert favorite_count to integer data type

#### Code

In [35]:
tweet_info_clean["favorite_count"] = tweet_info_clean.favorite_count.astype(int)

#### Test

In [36]:
tweet_info_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 4 columns):
tweet_id          2340 non-null object
timestamp         2340 non-null object
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
dtypes: int64(2), object(2)
memory usage: 73.2+ KB


### `tweet_info` should be a part of `twitter_archive`

#### Define

The columns "retweet_count" and "favorite_counts" should be a part of twitter_archive and these two can be merged on tweet_id

#### Code

In [37]:
twitter_clean.drop("timestamp",axis=1, inplace=True)
twitter_clean = twitter_clean.merge(tweet_info_clean, how="left", on="tweet_id")

#### Test

In [38]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
timestamp                     2340 non-null object
retweet_count          

### Sorting `twitter_archive` by timestamp

In [39]:
twitter_clean.sort_values(by=["timestamp"], inplace=True)

### One variable is present as 4 columns in `twitter_archive` table

#### Define

Create a new column to store all dog stages that a dog in a tweet belongs and drop the 4 columns names puppo, pupper, floofer and doggo

#### Code

In [40]:
def checkDogStage(a, b, c, d):
    no_nulls = 0
    output = ""
    for i in [a,b,c,d]:
        if i == None or i == "None":
            no_nulls += 1
        else:
            output = output + i + ", "
    if no_nulls == 4:
        return None
    else:
        return output.strip().strip(",")

twitter_clean["dog_stages"] = twitter_clean.apply(lambda row: checkDogStage(row.doggo, row.pupper, row.puppo, row.floofer), axis=1)
twitter_clean.drop(["puppo", "floofer", "pupper", "doggo"], axis=1, inplace=True)

### Test

In [41]:
twitter_clean.dog_stages.sample(10)

1937    pupper
158       None
885       None
1439      None
878       None
1116      None
1024      None
1327    pupper
968       None
1741      None
Name: dog_stages, dtype: object

In [44]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 1209 to 1726
Data columns (total 16 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
timestamp                     2340 non-null object
retweet_count                 2340 non-null float64
favorite_count                2340 non-null float64
dog_stages                    380 non-null object
dtypes: datetime64[ns](1), float64(6), int64(2), object(7)
memory usag

# Analyses

### Most commonly used words in tweets

In [ ]:
all_text = " ".join(rev for rev in twitter_clean.text)
stop_words = ["https", "co", "RT"] + list(STOPWORDS)
wordcloud = WordCloud(stopwords=stop_words, background_color="white").generate(all_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Number of images classified as dogs vs number of classified as not dogs in predictions

In [ ]:
pred_len = image_pred_clean.shape[0]
dogs_in_predictions = [image_pred_clean.p1_dog.sum(), image_pred_clean.p2_dog.sum(),image_pred_clean.p3_dog.sum()]
not_dog_predictions = [pred_len - image_pred_clean.p1_dog.sum(), pred_len - image_pred_clean.p2_dog.sum(), pred_len - image_pred_clean.p3_dog.sum()]

In [ ]:
dogs_in_predictions

In [ ]:
not_dog_predictions

In [ ]:
dog_nodog_labels = ["p1", "p2", "p3"]
dogs_bar = plt.bar(range(3), dogs_in_predictions)
nondogs_bar = plt.bar(range(3), not_dog_predictions, bottom=dogs_in_predictions)
plt.xticks(range(3), labels=dog_nodog_labels )
plt.xlabel("Prediction")
plt.ylabel("Number of Images classified as dogs or other")
plt.legend((dogs_bar[0], nondogs_bar[0]), ('Classified as dogs', 'Classified as other'))
plt.title("Images classified as dogs or other in each prediction")
plt.show()

### Average confidence level in predictions 

In [ ]:
conf_levels = [image_pred_clean.p1_conf.mean(), image_pred_clean.p2_conf.mean(), image_pred_clean.p3_conf.mean()]
plt.bar(range(3), conf_levels)
plt.xlabel("Prediction")
plt.ylabel("Confidence level")
plt.title("Average confidence levels of each prediction")
plt.show()

The confidence level of predictions seem to be high for the first prediction and it decreases with further predictions.

In [ ]:
non_null = twitter_clean[~twitter_clean.timestamp.isnull()]
plt.plot(non_null.timestamp, non_null.retweet_count)
plt.show()

### Does rating affect likes?

In [ ]:
import statsmodels.api as sm
twitter_clean["intercept"] = 1
test = twitter_clean[~twitter_clean.favorite_count.isnull()]
fav_model = sm.OLS(test["favorite_count"], test[["intercept", "rating_numerator"]])
fav_model_results = fav_model.fit()
fav_model_results.summary()

From the above model, it is evident that the rating of a dog does not affect the number of likes that a tweet receives from the p-value. 